In [1]:
import sys
import os
import csv
import time
import logging as log
from pyspark import SparkContext, SparkConf, Row
from pyspark.sql import SQLContext, HiveContext, SparkSession
import getpass
from datetime import timedelta, date
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *
from subprocess import call
import pandas as pd
import functools

In [2]:
#loading all the datasets in spark dataframe

df_links = spark.read.csv('links.csv', header=True)
df_links.show(5)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows



In [3]:
df_movies = spark.read.csv('movies.csv', header=True)
df_movies.show(truncate = False)
df_movies.count()

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

58098

In [4]:
df_movies.select('title').distinct().count()

58020

In [5]:
df_ratings = spark.read.csv('ratings.csv', header=True)
df_ratings.show(truncate = False)

+------+-------+------+----------+
|userId|movieId|rating|timestamp |
+------+-------+------+----------+
|1     |307    |3.5   |1256677221|
|1     |481    |3.5   |1256677456|
|1     |1091   |1.5   |1256677471|
|1     |1257   |4.5   |1256677460|
|1     |1449   |4.5   |1256677264|
|1     |1590   |2.5   |1256677236|
|1     |1591   |1.5   |1256677475|
|1     |2134   |4.5   |1256677464|
|1     |2478   |4.0   |1256677239|
|1     |2840   |3.0   |1256677500|
|1     |2986   |2.5   |1256677496|
|1     |3020   |4.0   |1256677260|
|1     |3424   |4.5   |1256677444|
|1     |3698   |3.5   |1256677243|
|1     |3826   |2.0   |1256677210|
|1     |3893   |3.5   |1256677486|
|2     |170    |3.5   |1192913581|
|2     |849    |3.5   |1192913537|
|2     |1186   |3.5   |1192913611|
|2     |1235   |3.0   |1192913585|
+------+-------+------+----------+
only showing top 20 rows



In [29]:
df_tags = spark.read.csv('tags.csv', header=True)
df_tags.show(truncate = False)

+------+-------+--------------+----------+
|userId|movieId|tag           |timestamp |
+------+-------+--------------+----------+
|14    |110    |epic          |1443148538|
|14    |110    |Medieval      |1443148532|
|14    |260    |sci-fi        |1442169410|
|14    |260    |space action  |1442169421|
|14    |318    |imdb top 250  |1442615195|
|14    |318    |justice       |1442615192|
|14    |480    |Dinosaurs     |1443148563|
|14    |593    |psychothriller|1444014286|
|14    |1682   |philosophy    |1442615158|
|14    |1682   |surveillance  |1442615167|
|14    |7458   |Epic          |1443148675|
|14    |95311  |Pixar         |1443148494|
|14    |117529 |dinosaurs     |1443148646|
|27    |260    |classic sci-fi|1440448113|
|27    |260    |Harrison Ford |1440448123|
|27    |260    |must see      |1440448132|
|27    |260    |sci-fi        |1440448094|
|42    |37733  |disappointing |1264106059|
|42    |37733  |overrated     |1264106052|
|42    |37733  |stupid        |1264106067|
+------+---

In [15]:
#creating a temporary table to use SparkSQL to query results

df_ratings.registerTempTable('rating_table')
df_movies.registerTempTable('movies_table')
df_tags.registerTempTable('tags_table')

In [34]:
#Showing all popularity based on the count of users who have watched the show and sorting based on the popularity
sqlContext.sql("select r.movieId, title, count(distinct userId) as no_of_users_watched from rating_table r join movies_table m on r.movieId = m.movieId group by r.movieId , title order by no_of_users_watched desc").show(truncate = False)

+-------+------------------------------------------------------------------------------+-------------------+
|movieId|title                                                                         |no_of_users_watched|
+-------+------------------------------------------------------------------------------+-------------------+
|318    |Shawshank Redemption, The (1994)                                              |97999              |
|356    |Forrest Gump (1994)                                                           |97040              |
|296    |Pulp Fiction (1994)                                                           |92406              |
|593    |Silence of the Lambs, The (1991)                                              |87899              |
|2571   |Matrix, The (1999)                                                            |84545              |
|260    |Star Wars: Episode IV - A New Hope (1977)                                     |81815              |
|480    |Jurassic P

In [35]:
#Showing 10 most popular genres of all time
sqlContext.sql("select r.movieId, genres, count(distinct userId) as no_of_users_watched from rating_table r join movies_table m on r.movieId = m.movieId group by r.movieId , genres order by no_of_users_watched desc").show(10,truncate = False)

+-------+-------------------------------------------+-------------------+
|movieId|genres                                     |no_of_users_watched|
+-------+-------------------------------------------+-------------------+
|318    |Crime|Drama                                |97999              |
|356    |Comedy|Drama|Romance|War                   |97040              |
|296    |Comedy|Crime|Drama|Thriller                |92406              |
|593    |Crime|Horror|Thriller                      |87899              |
|2571   |Action|Sci-Fi|Thriller                     |84545              |
|260    |Action|Adventure|Sci-Fi                    |81815              |
|480    |Action|Adventure|Sci-Fi|Thriller           |76451              |
|527    |Drama|War                                  |71516              |
|110    |Action|Drama|War                           |68803              |
|1      |Adventure|Animation|Children|Comedy|Fantasy|68469              |
+-------+-----------------------------

In [37]:
#Showing 10 least popular movies of all time
sqlContext.sql('select movieId, title ,no_of_users_watched from (select r.movieId ,title , count(distinct userId) as no_of_users_watched from rating_table r join movies_table m on r.movieId = m.movieId group by r.movieId, title) moviecount order by no_of_users_watched asc').show(10, truncate=False)

+-------+-----------------------------------+-------------------+
|movieId|title                              |no_of_users_watched|
+-------+-----------------------------------+-------------------+
|177713 |Absurd Encounter with Fear (1967)  |1                  |
|159749 |Bumblefuck, USA (2011)             |1                  |
|136206 |Ninja Academy (1988)               |1                  |
|180483 |Le potager de mon grand-père (2016)|1                  |
|148913 |Die Akte Golgatha (2010)           |1                  |
|154240 |All About Love (2001)              |1                  |
|165513 |Magic Kitchen (2004)               |1                  |
|189377 |Pappa pia (2017)                   |1                  |
|112458 |Son-Daughter, The (1932)           |1                  |
|133511 |I Will, I Will...For Now (1976)    |1                  |
+-------+-----------------------------------+-------------------+
only showing top 10 rows

